In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt 

In [2]:
"""

  Fashion-MNSIT의 파라미터 
  
  root(string) --> 데이터 셋의 디렉토리
  train(bool, optional) --> True : training.pt에서 데이터 셋 생성 False : test.pt
  download(bool, optional) --> True: root 디렉토리에 데이터셋을 인터넷에서 다운받음
  transform (callable, optional) --> PIL 이미지를 변형된 버젼으로 반환
  target_transform (callable, optional) --> A function/transform that takes in the target and transforms it

"""
# 오픈 데이터 셋의 학습 데이터 다운로드 
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download = True,
    transform = ToTensor(),
)

# 오픈 데이터 셋의 테스트 데이터 다운로드
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
batch_size = 64
# batch size = 하나의 batch에 학습 데이터의 수

# 데이터 로더 생성
train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## 모델 생성

In [4]:
# 트레이닝을 위한 cpu 또는 gpu 
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [12]:
# 모델 정의 
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
# loss function과 optimizer 정의 
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # 에러 예측 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파 (Backpropagation)
    # 출력층에서 제시한 값이 실제 원하는 값에 가까워지도록 학습하기 위한 통계적 방법
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 10
# epoch : 한번의 epoch는 인공 신경망에서 전체 데이터 셋에 대해 forward pass/ backward pass 과정을 거친 것을 말함.
#       : 전체 데이터 셋에 대해 한번 학습을 완료한 상태 

for t in range(epochs) : 
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader, model, loss, optimizer)
  test(test_dataloader, model, loss)
print("Done!")

Epoch 1
---------------------------
loss: 1.143489 [    0/60000]
loss: 1.150624 [ 6400/60000]
loss: 0.972987 [12800/60000]
loss: 1.115982 [19200/60000]
loss: 0.973370 [25600/60000]
loss: 1.009588 [32000/60000]
loss: 1.046803 [38400/60000]
loss: 0.991352 [44800/60000]
loss: 1.037433 [51200/60000]
loss: 0.959822 [57600/60000]
Test Error: 
 Accuracy: 66.2%, Avg loss: 0.975282 

Epoch 2
---------------------------
loss: 1.025757 [    0/60000]
loss: 1.053620 [ 6400/60000]
loss: 0.859803 [12800/60000]
loss: 1.026929 [19200/60000]
loss: 0.885736 [25600/60000]
loss: 0.914547 [32000/60000]
loss: 0.970923 [38400/60000]
loss: 0.917400 [44800/60000]
loss: 0.960896 [51200/60000]
loss: 0.893675 [57600/60000]
Test Error: 
 Accuracy: 67.3%, Avg loss: 0.903451 

Epoch 3
---------------------------
loss: 0.939514 [    0/60000]
loss: 0.986302 [ 6400/60000]
loss: 0.779019 [12800/60000]
loss: 0.964839 [19200/60000]
loss: 0.827518 [25600/60000]
loss: 0.845025 [32000/60000]
loss: 0.917519 [38400/60000]
loss:

In [19]:
# model 세이브
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


# 모델 불러오기

In [21]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [22]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [28]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: {actual}"')

Predicted: "Ankle boot", Actual: Ankle boot"
